https://www.gymlibrary.dev/environments/classic_control/cart_pole/

# Import dependencies

In [1]:
!pip install stable-baselines3[extra]

# Load the environment

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [4]:
environmentName = 'CartPole-v1'
env = gym.make(environmentName, render_mode='human')

In [5]:
environmentName

'CartPole-v1'

In [6]:
for episode in range(1, 11):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        step_result = env.step(action)
        
        if len(step_result) == 4:
            n_state, reward, done, info = step_result
        elif len(step_result) == 5:
            n_state, reward, done, truncated, info = step_result
            done = done or truncated

        score += reward
    print('Episode: {} Score: {}'.format(episode, score))
env.close()

c:\Users\csang\miniconda3\envs\fhgr\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 1 Score: 17.0
Episode: 2 Score: 17.0
Episode: 3 Score: 14.0
Episode: 4 Score: 10.0
Episode: 5 Score: 15.0
Episode: 6 Score: 12.0
Episode: 7 Score: 15.0
Episode: 8 Score: 45.0
Episode: 9 Score: 15.0
Episode: 10 Score: 10.0


In [8]:
env.action_space

Discrete(2)

In [9]:
env.action_space.sample()

1

In [10]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)